In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import pandas as pd
from dotenv import dotenv_values
config = dotenv_values(".env")
persistent_directory = "db/incidents"


In [18]:
def ingest(incident_file_path="data/incidents.csv"):
    df = pd.read_csv(incident_file_path)
    incidents = df["description"].tolist()
    metadatas = df.to_dict(orient='records')
    embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_API_KEY"], model='text-embedding-3-large')
    Chroma.from_texts(incidents, embedding = embeddings, metadatas=metadatas, persist_directory=persistent_directory)
ingest()

In [21]:
def retrive(incident_description):
    embeddings = OpenAIEmbeddings(api_key=config["OPEN_AI_API_KEY"], model='text-embedding-3-large')
    db = Chroma(persist_directory=persistent_directory,
                embedding_function=embeddings)
    retriever = db.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 3, "score_threshold": 0.7},
    )
    relevant_incidents = retriever.invoke(incident_description)
    return relevant_incidents
relevant_incidents = retrive("Wrong feature flag set for Reporting; users in us-west-2 see incomplete menu.")

In [25]:
contexts = []
for relevant_incident in relevant_incidents:
    contexts.append(
    {
        'description':relevant_incident.metadata['description'],
        'solution':relevant_incident.metadata['solution']
    })

In [26]:
contexts

[{'description': 'Wrong feature flag set for Reporting; users in us-west-2 see incomplete menu.',
  'solution': 'Restored feature flag defaults per environment; added change control.'},
 {'description': 'Wrong feature flag set for Reporting; users in us-west-2 see incomplete menu.',
  'solution': 'Restored feature flag defaults per environment; added change control.'},
 {'description': 'Wrong feature flag set for Reporting; users in us-west-2 see incomplete menu.',
  'solution': 'Restored feature flag defaults per environment; added change control.'}]